Lets verify data split

In [1]:
from vpr_model import VPRModel
from dataloaders.MapsDataloader import MapsDataModule

import pandas as pd
import os
from PIL import Image
import numpy as np
import rasterio
from rasterio.windows import Window
import matplotlib.pyplot as plt
from pandas import DataFrame
import datetime

import utm

In [15]:

visloc_satelite_shandan_output_csv = '/workspace/repos/drone-loc-no-gps/Dataframes/Shandan.csv'
visloc_taizhou = '/home/user/PycharmProjects/mgr-repack/Dataframes/Taizhou-1.csv'
visloc_taizhou_out = '/home/user/PycharmProjects/mgr-repack/Dataframes/Taizhou-1_out.csv'
df = pd.read_csv(visloc_taizhou)

In [16]:
def get_utm(row):
    e, n, zone_num, zone_let = utm.from_latlon(row.lat, row.lon)
    zone = f'{zone_num}{zone_let or ""}'

    return {'e_utm': e, 'n_utm': n, 'zone_utm': zone}

In [ ]:
df = df.join(df.apply(get_utm, axis=1).apply(pd.Series))

In [ ]:
# APPLY IS NOT REQUIRED
df.to_csv(visloc_taizhou_out, index=True)
df = pd.read_csv(visloc_taizhou_out)

In [ ]:
# satellite_records = df[df['friendly-name'].str.contains('satellite')]
satellite_records = df[~df['friendly-name'].str.contains('uav')]
uav_records = df[df['friendly-name'].str.contains('uav')]

In [23]:
common_place_ids = set(satellite_records['place_id']).intersection(set(uav_records['place_id']))
if not common_place_ids:
    print("The same place_id for satellite and uav not found")

In [24]:
first_common_place_id = sorted(common_place_ids)[0]

In [ ]:
satellite_series = satellite_records[satellite_records['place_id'] == first_common_place_id]
uav_series = uav_records[uav_records['place_id'] == first_common_place_id]

print(f"Liczba wierszy w satellite_series: {len(satellite_series)}")
if len(satellite_series) == 0:
    print("Błąd: satellite_series jest pusty! Nie znaleziono place_id.")
elif len(satellite_series) > 1:
    print("Ostrzeżenie: satellite_series ma więcej niż jeden wiersz. Użycie .item() jest ryzykowne.")
    print(satellite_series)

Liczba wierszy w satellite_series: 1


In [51]:
def get_neigbourhood(series, df: DataFrame, range: int = 100):
    if isinstance(series, pd.DataFrame):
        if len(series) == 0:
            return pd.DataFrame()
        series = series.iloc[0]
    
    e_center = series['e_utm']
    n_center = series['n_utm'] 
    zone = series['zone_utm']
    
    neighbours = df[(
        df['e_utm'].between(e_center-range, e_center+range) &
        df['n_utm'].between(n_center-range, n_center+range) &
        (df['zone_utm'] == zone)
        )]
    return neighbours

In [52]:
satellite_related = get_neigbourhood(satellite_series, df)
print(len(satellite_related))
uav_related = get_neigbourhood(uav_series, df)
print(len(uav_related))

12
12
